## Demande de viz complémentaire : 
#### en réponse au commentaire du 20/05 de Guillaume Daudin (doc Etape 3, Axe 1)
"il faudrait refaire la viz [Proportion type de produits exportés par direction pour 1789] en excluant les exportations vers les ports francs? Et peut-être pour 1750 ? Et en classant par La Rochelle ?"

#### Méthode adoptée : 
- reprise du code de réalisation de la viz lors du sprint
- on enlève dans les données les flux avec pour attribut 'partner_grouping' = 'France' (ports francs)
- on réorganise le jeu de données pour que les produits les plus exportés par la DFLR apparaissent eu haut du diagramme => pour l'instant je n'y arrive pas

In [ ]:
from poitousprint import Toflit
from vega import VegaLite
import pandas as pd
from operator import itemgetter # to sort lists

toflit_client = Toflit()

In [ ]:
def prepare_flow(flow):
    f = flow.copy()
    if f['export_import'] == 'Imports' or f['export_import'] == 'import':
        f['export_import'] = 'Imports'
    if f['customs_region'] == '' or f['customs_region'] == 'National':
        f['customs_region_simpl'] = 'National'
    elif f['customs_region'] == 'La Rochelle':
        f['customs_region_simpl'] = 'La Rochelle'
    else:
        f['customs_region_simpl'] = 'Autre direction'
    f['value'] = float(f['value']) if f['value'] != '' else 0
    return f



flows_1789 = toflit_client.get_flows(year=1789, best_guess_region_prodxpart='1')
flows_1750  = toflit_client.get_flows(year=1750, best_guess_region_prodxpart='1')

total_exports_per_direction = {}
total_imports_per_direction = {}

for f in flows_1789:
    flow = prepare_flow(f)
    if flow['export_import'] == 'Imports':
        if flow['customs_region_simpl'] not in total_imports_per_direction:
            total_imports_per_direction[flow['customs_region_simpl']] = 0
        total_imports_per_direction[flow['customs_region_simpl']] += flow['value']
    else:
        if flow['customs_region_simpl'] not in total_exports_per_direction:
            total_exports_per_direction[flow['customs_region_simpl']] = 0
        total_exports_per_direction[flow['customs_region_simpl']] += flow['value']
        
def clean_flow(flow):
    f = flow.copy()
    abs_map = total_exports_per_direction if f['export_import'] == 'Exports' else total_imports_per_direction
    f['value_rel_per_direction'] = f['value'] / abs_map[f['customs_region_simpl']]
    return f       

flows_1789 = [clean_flow(prepare_flow(f)) for f in flows_1789]
flows_1750 = sorted([clean_flow(prepare_flow(f)) for f in flows_1750], key=itemgetter('value'))

In [ ]:
VegaLite({
    "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
    "title": "Proportion des types de produits exportés par direction pour 1789",
    "mark": "circle",
    "encoding": {
        "x": {
            "field": "customs_region_simpl",
            "type": "nominal", 
            "title": "Direction",
            "sort": "-size"
        },
        "y": {
            "type": "nominal",
            "field": "product_revolutionempire",
            "sort": "-size"
        },
        "color": {
            "field": "customs_region_simpl",
            "type": "nominal", 
            "title": "Direction",
            "sort": "-size"
        },
        "size": {
            "type": "quantitative",
            "field": "value_rel_per_direction",
            "title": "Part de la somme des valeurs par rapport au total par direction",
            "aggregate": "sum"
        }
    }
}, pd.DataFrame([clean_flow(f) for f in flows_1789 if (f['export_import'] == 'Exports' and f['partner_grouping'] != 'France')]))

In [ ]:
VegaLite({
    "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
    "title": "Proportion des types de produits exportés par direction pour 1750",
    "mark": "circle",
    "encoding": {
        "x": {
            "field": "customs_region_simpl",
            "type": "nominal", 
            "title": "Direction",
            "sort": "-size"
        },
        "y": {
            "type": "nominal",
            "field": "product_revolutionempire",
            "sort": "-size"
        },
        "color": {
            "field": "customs_region_simpl",
            "type": "nominal", 
            "title": "Direction",
            "sort": ["Étoffes de laine", "Toiles de chanvre et de lin", "Sucre", "Vins de Bordeaux","Étoffes de soie", "Café", "Mercerie", "Indigo", "Ouvrages divers et mélangés", "Étoffes diverses", "Dorure", "Toiles diverses", "Toiles de coton", "Dentelle", "Soieries diverses", "Eaux-de-vie et liqueurs"]
        },
        "size": {
            "type": "quantitative",
            "field": "value_rel_per_direction",
            "title": "Part de la somme des valeurs par rapport au total par direction",
            "aggregate": "sum"
        }
    }
}, pd.DataFrame([clean_flow(f) for f in flows_1750 if (f['export_import'] == 'Exports' and f['partner_grouping'] != 'France')]))

In [ ]:
VegaLite({
    "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
    "title": "Proportion des types de produits exportés par direction pour 1789",
    "mark": "circle",
    "encoding": {
        "column": {
            "field": "customs_region_simpl"
        },
        "x": {
            "field": "partner_grouping",
            "type": "nominal", 
            "title": "Partenaire de commerce",
            "sort": "-size"
        },
        "y": {
            "type": "nominal",
            "field": "product_revolutionempire",
            "sort": "-size"
        },
        "color": {
            "field": "customs_region_simpl",
            "title": "Comparaison entre la Rochelle et les autres directions",
            "type": "nominal", 
            "title": "Direction",
            "sort": "-size"
        },
        "size": {
            "type": "quantitative",
            "field": "value_rel_per_direction",
            "aggregate": "sum"
        }
    }
}, pd.DataFrame([clean_flow(f) for f in flows_1789 if (f['export_import'] == 'Exports' and f['partner_grouping'] != 'France')]))

In [ ]:
VegaLite({
    "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
    "title": "Proportion des types de produits exportés par direction pour 1750",
    "mark": "circle",
    "encoding": {
        "column": {
            "field": "customs_region_simpl"
        },
        "x": {
            "field": "partner_grouping",
            "type": "nominal", 
            "title": "Partenaire de commerce",
            "sort": "-size"
        },
        "y": {
            "type": "nominal",
            "field": "product_revolutionempire",
            "sort": "-size"
        },
        "color": {
            "field": "customs_region_simpl",
            "title": "Comparaison entre la Rochelle et les autres directions",
            "type": "nominal", 
            "title": "Direction",
            "sort": "-size"
        },
        "size": {
            "type": "quantitative",
            "field": "value_rel_per_direction",
            "aggregate": "sum"
        }
    }
}, pd.DataFrame([clean_flow(f) for f in flows_1750 if (f['export_import'] == 'Exports' and f['partner_grouping'] != 'France')]))